## ETL

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
# Caminho do arquivo
file_path = "/content/Base-Dados-Desafio-500k.xlsx"

# Leitura das abas
df_vendas = pd.read_excel(file_path, sheet_name=0)
df_produtos = pd.read_excel(file_path, sheet_name=1)

print("Vendas:", df_vendas.shape)
print("Produtos:", df_produtos.shape)


Vendas: (500000, 7)
Produtos: (5, 2)


##Exploração Inicial dos Dados (EDA)



In [ ]:
df_vendas.head()


,CLIENTE,IDADE,ESTADO,PRODUTO,QUANTIDADE_VENDIDA,PREÇO_UNITARIO,DATA
0,9010.0,72.0,SANTA CATARINA,E,243,50,2024-10-26
1,63611.0,46.0,PARANÁ,B,320,15,2024-12-27
2,56708.0,67.0,BAHIA,D,143,30,2023-06-30
3,3924.0,46.0,SÃO PAULO,B,295,15,2025-06-02
4,47317.0,53.0,RIO DE JANEIRO,C,203,45,2025-02-15


In [ ]:
df_vendas.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 7 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   CLIENTE             499484 non-null  float64       
 1   IDADE               499850 non-null  float64       
 2   ESTADO              499630 non-null  object        
 3   PRODUTO             499452 non-null  object        
 4   QUANTIDADE_VENDIDA  500000 non-null  int64         
 5   PREÇO_UNITARIO      500000 non-null  int64         
 6   DATA                500000 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(2), object(2)
memory usage: 26.7+ MB


In [ ]:
df_vendas.describe()


,CLIENTE,IDADE,QUANTIDADE_VENDIDA,PREÇO_UNITARIO,DATA
count,499484.000000,499850.000000,500000.000000,500000.000000,500000
mean,40015.170138,39.183431,185.928544,32.980870,2023-07-02 21:50:55.795199744
min,1.000000,18.000000,26.000000,0.000000,2021-01-01 00:00:00
25%,19993.000000,31.000000,147.000000,25.000000,2022-04-03 00:00:00
50%,40052.500000,39.000000,187.000000,30.000000,2023-07-02 00:00:00
75%,59981.000000,47.000000,231.000000,45.000000,2024-10-01 00:00:00
max,80000.000000,80.000000,432.000000,50.000000,2025-12-31 00:00:00
std,23088.348541,11.560814,80.424516,12.914743,NaN


In [ ]:
# colunas
df_vendas.columns


Index(['CLIENTE', 'IDADE', 'ESTADO', 'PRODUTO', 'QUANTIDADE_VENDIDA',
       'PREÇO_UNITARIO', 'DATA'],
      dtype='object')

##Data Quality

- A base contém aproximadamente 500 mil registros de vendas, com dados entre janeiro/2021 e dezembro/2025.

- As variáveis numéricas apresentam comportamento consistente, sem valores negativos.

- A idade dos clientes varia entre 18 e 80 anos, todos int, com média em torno de 39 anos. Alguns valores nulos/NaN, porém nenhum zero.

- A quantidade vendida não apresenta valores negativos ou fora de um intervalo esperado [26 : 431]. Porém apresenta alguns valores nulos e 0.

- O preço unitário varia entre R$ 15 e R$ 50. Apresenta alguns valores nulos e 0.

- A base apresentou Valores Nulos em Colunas Críticas como CLIENTE, PRODUTO e PREÇO UNITÁRIO.

- ESTADO e PRODUTO são carregados com o tipo Object

- FATURAMENTO apresenta dados float com duas casas decimais. Apresenta valores 0 na amostragem.
-FATURAMENTO  não apresentou valores outliers usando a metodologia do IQR. Tendo variação de [ 0 : 15850 ]

### Nulos

In [ ]:
#Valores nulos

df_vendas.isnull().sum().sort_values(ascending=False)


,0
PRODUTO,548
CLIENTE,516
ESTADO,370
IDADE,150
QUANTIDADE_VENDIDA,0
PREÇO_UNITARIO,0
DATA,0


### Duplicados

In [ ]:
#Linhas duplicadas
df_vendas.duplicated().sum()
df_vendas[df_vendas.duplicated()]


,CLIENTE,IDADE,ESTADO,PRODUTO,QUANTIDADE_VENDIDA,PREÇO_UNITARIO,DATA
240349,78462.0,52.0,SÃO PAULO,D,171,30,2024-09-29
346732,59514.0,36.0,BAHIA,D,126,30,2025-02-18
445556,20935.0,61.0,RIO DE JANEIRO,A,78,25,2024-10-25


### Preços

In [ ]:
#Tipos inconsistentes
df_vendas.dtypes


,0
CLIENTE,float64
IDADE,float64
ESTADO,object
PRODUTO,object
QUANTIDADE_VENDIDA,int64
PREÇO_UNITARIO,int64
DATA,datetime64[ns]


In [ ]:
#Valores inválidos
# Quantidade ou preço menores ou iguais a zero
df_vendas[
    (df_vendas["QUANTIDADE_VENDIDA"] <= 0) |
    (df_vendas["PREÇO_UNITARIO"] <= 0)
]


,CLIENTE,IDADE,ESTADO,PRODUTO,QUANTIDADE_VENDIDA,PREÇO_UNITARIO,DATA
253,75902.0,19.0,RIO GRANDE DO SUL,NaN,138,0,2023-05-05
254,64008.0,35.0,RIO DE JANEIRO,NaN,158,0,2022-02-28
255,17456.0,21.0,SÃO PAULO,NaN,148,0,2025-05-25
256,78946.0,35.0,BAHIA,NaN,137,0,2024-05-25
257,23482.0,42.0,PARANÁ,NaN,160,0,2024-10-08
...,...,...,...,...,...,...,...
796,2366.0,28.0,RIO DE JANEIRO,NaN,204,0,2022-06-16
797,64065.0,44.0,RIO DE JANEIRO,NaN,329,0,2024-10-31
798,3952.0,36.0,SÃO PAULO,NaN,238,0,2021-01-03
799,73634.0,36.0,PARAÍBA,NaN,286,0,2022-10-08


### Datas

In [ ]:
#Datas inválidas
df_vendas[
    df_vendas["DATA"].isna()
]


,CLIENTE,IDADE,ESTADO,PRODUTO,QUANTIDADE_VENDIDA,PREÇO_UNITARIO,DATA


In [ ]:
# Range de datas
data_min = df_vendas["DATA"].min()
data_max = df_vendas["DATA"].max()

data_min, data_max


(Timestamp('2021-01-01 00:00:00'), Timestamp('2025-12-31 00:00:00'))

### Idade

In [ ]:
# Estatísticas básicas da idade
df_vendas["IDADE"].describe()


,IDADE
count,499850.000000
mean,39.183431
std,11.560814
min,18.000000
25%,31.000000
50%,39.000000
75%,47.000000
max,80.000000


In [ ]:
# Idades fora de um range razoável (ex: < 0 ou > 120)
idades_invalidas = df_vendas[
    (df_vendas["IDADE"] < 0) | (df_vendas["IDADE"] > 120)
]

print(f"Idades fora do padrão: {idades_invalidas.shape[0]}")
idades_invalidas[["CLIENTE", "IDADE"]]


Idades fora do padrão: 0


,CLIENTE,IDADE


### Vendas e Faturamento

In [ ]:
# Verificar tipos das colunas críticas
df_vendas[["QUANTIDADE_VENDIDA", "PREÇO_UNITARIO"]].dtypes


,0
QUANTIDADE_VENDIDA,int64
PREÇO_UNITARIO,int64


In [ ]:
# Forçar conversão e identificar valores inválidos
df_vendas["QUANTIDADE_VENDIDA"] = pd.to_numeric(
    df_vendas["QUANTIDADE_VENDIDA"], errors="coerce"
)

df_vendas["PREÇO_UNITARIO"] = pd.to_numeric(
    df_vendas["PREÇO_UNITARIO"], errors="coerce"
)


In [ ]:
# Linhas onde a conversão falhou
valores_invalidos = df_vendas[
    df_vendas["QUANTIDADE_VENDIDA"].isna() |
    df_vendas["PREÇO_UNITARIO"].isna()
]

print(f"Registros com valores não numéricos: {valores_invalidos.shape[0]}")


Registros com valores não numéricos: 0


In [ ]:
# Identificar preços com mais de duas casas decimais
precos_invalidos = df_vendas[
    (df_vendas["PREÇO_UNITARIO"] * 100) % 1 != 0
]

print(f"Preços com mais de duas casas decimais: {precos_invalidos.shape[0]}")
precos_invalidos[["PREÇO_UNITARIO"]].head()


Preços com mais de duas casas decimais: 0


,PREÇO_UNITARIO


In [ ]:
# Garantir que a coluna está em formato datetime
df_vendas["DATA"] = pd.to_datetime(df_vendas["DATA"], errors="coerce")


In [ ]:
# Faturamento por venda
df_fat = df_vendas[["QUANTIDADE_VENDIDA", "PREÇO_UNITARIO"]].copy()
df_fat["FATURAMENTO"] = (
    df_fat["QUANTIDADE_VENDIDA"] * df_fat["PREÇO_UNITARIO"]
)
df_fat["FATURAMENTO"].describe()


,FATURAMENTO
count,500000.000000
mean,6025.757500
std,3269.301248
min,0.000000
25%,3900.000000
50%,5100.000000
75%,8955.000000
max,15850.000000


In [ ]:
# Cálculo dos quartis
Q1 = df_fat["FATURAMENTO"].quantile(0.25)
Q3 = df_fat["FATURAMENTO"].quantile(0.75)
IQR = Q3 - Q1

# Limites para outliers
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

limite_inferior, limite_superior


(np.float64(-3682.5), np.float64(16537.5))

In [ ]:
outliers_faturamento = df_fat[
    (df_fat["FATURAMENTO"] < limite_inferior) |
    (df_fat["FATURAMENTO"] > limite_superior)
]

outliers_faturamento.shape[0]

0

##Limpeza e Padronização

- Remoção de registros sem cliente
Justificativa: inviabilizam análises de comportamento, recorrência e perfil demográfico.

- Remoção de idades nulas ou fora do intervalo válido (0–120 anos)
Justificativa: evita distorções em análises por faixa etária.

- Remoção de registros sem estado
Justificativa: comprometeriam análises regionais, um dos eixos do negócio.

- Remoção de vendas sem produto ou categoria associada
Justificativa: impossibilitam análises por categoria, considerada dimensão central do negócio.

Tratamento de tipos de dados:

- Conversão de datas para datetime

- Conversão de quantidade e preço para numérico

- Padronização textual de estado, produto e categoria

- Criação da métrica de faturamento (valor_total)
Justificativa: permitir análises financeiras e identificação de vendas atípicas.

In [ ]:
#Padronizar nomes das colunas
df_vendas.columns = (
    df_vendas.columns
    .str.lower()
    .str.strip()
)


In [ ]:
#Padronizar textos (estado, produto)
df_vendas["estado"] = (
    df_vendas["estado"]
    .str.upper()
    .str.strip()
)

df_vendas["produto"] = (
    df_vendas["produto"]
    .str.upper()
    .str.strip()
)


In [ ]:
#Converter tipos
df_vendas["data"] = pd.to_datetime(df_vendas["data"], errors="coerce")


In [ ]:
#Remover duplicados
df_vendas = df_vendas.drop_duplicates()


In [ ]:
#Criar coluna de valor total
df_vendas.loc[:, "valor_total"] = (
    df_vendas["quantidade_vendida"] * df_vendas["preço_unitario"]
)



In [ ]:
#Enriquecimento com Dicionário de Produtos (Fonte da Verdade)
df_produtos.columns = (
    df_produtos.columns
    .str.lower()
    .str.strip()
)

df_produtos["produto"] = df_produtos["produto"].str.upper().str.strip()
df_produtos["categoria"] = df_produtos["categoria"].str.upper().str.strip()


In [ ]:
df_vendas = df_vendas.merge(
    df_produtos,
    on="produto",
    how="left"
)


In [ ]:
#Validar produtos sem categoria

df_vendas[df_vendas["categoria"].isna()]


,cliente,idade,estado,produto,quantidade_vendida,preço_unitario,data,valor_total,categoria
253,75902.0,19.0,RIO GRANDE DO SUL,NaN,138,0,2023-05-05,0,NaN
254,64008.0,35.0,RIO DE JANEIRO,NaN,158,0,2022-02-28,0,NaN
255,17456.0,21.0,SÃO PAULO,NaN,148,0,2025-05-25,0,NaN
256,78946.0,35.0,BAHIA,NaN,137,0,2024-05-25,0,NaN
257,23482.0,42.0,PARANÁ,NaN,160,0,2024-10-08,0,NaN
...,...,...,...,...,...,...,...,...,...
796,2366.0,28.0,RIO DE JANEIRO,NaN,204,0,2022-06-16,0,NaN
797,64065.0,44.0,RIO DE JANEIRO,NaN,329,0,2024-10-31,0,NaN
798,3952.0,36.0,SÃO PAULO,NaN,238,0,2021-01-03,0,NaN
799,73634.0,36.0,PARAÍBA,NaN,286,0,2022-10-08,0,NaN


In [ ]:
## Removendo registros sem produto identificado,
# pois não é possível associar categoria nem gerar métricas de negócio
df_vendas = df_vendas[df_vendas["produto"].notna()].copy()


In [ ]:
#Remover preços e quantidades inválidas
df_vendas = df_vendas[
    (df_vendas["quantidade_vendida"] > 0) &
    (df_vendas["preço_unitario"] > 0)
].copy()


In [ ]:
# Remover registros sem identificação de cliente,
# pois não permitem análises de comportamento ou segmentação
df_vendas = df_vendas[df_vendas["cliente"].notna()].copy()


In [ ]:
# % de dados faltantes
(df_vendas.isnull().mean() * 100).round(2)


,0
cliente,0.00
idade,0.03
estado,0.07
produto,0.00
quantidade_vendida,0.00
preço_unitario,0.00
data,0.00
valor_total,0.00
categoria,0.00


## Resumo final da Base Tratada

- A base final contém ~499 mil registros de vendas válidas, após aplicação de regras de qualidade e consistência.

- O período analisado vai de janeiro/2021 a dezembro/2025, garantindo cobertura histórica suficiente para análises de tendência e sazonalidade.

- O perfil dos clientes concentra-se em adultos, com idade média de ~39 anos, variando entre 18 e 80 anos.
<br>

---
<br>

O faturamento por venda apresenta:

- mediana em torno de R$ 5.100

- 75% das vendas abaixo de ~R$ 9.000

- valores máximos próximos de R$ 15.800, caracterizando vendas atípicas (outliers).

- A categoria mais recorrente é Utilidades Domésticas, indicando maior volume de vendas nesse segmento.

- Após o tratamento, não há valores nulos nas colunas críticas para análise (produto, categoria, preço, quantidade e data).



In [ ]:
# Resumo final
df_vendas.describe(include="all")


,cliente,idade,estado,produto,quantidade_vendida,preço_unitario,data,valor_total,categoria
count,498933.000000,498783.000000,498563,498933,498933.000000,498933.000000,498933,498933.000000,498933
unique,NaN,NaN,7,5,NaN,NaN,NaN,NaN,5
top,NaN,NaN,SÃO PAULO,C,NaN,NaN,NaN,NaN,UTILIDADES DOMÉSTICAS
freq,NaN,NaN,160929,100246,NaN,NaN,NaN,NaN,100246
mean,40014.885289,39.183378,NaN,NaN,185.922907,33.016758,2023-07-02 22:06:25.085773568,6032.178970,NaN
min,1.000000,18.000000,NaN,NaN,26.000000,15.000000,2021-01-01 00:00:00,650.000000,NaN
25%,19992.000000,31.000000,NaN,NaN,147.000000,25.000000,2022-04-03 00:00:00,3900.000000,NaN
50%,40053.000000,39.000000,NaN,NaN,187.000000,30.000000,2023-07-02 00:00:00,5100.000000,NaN
75%,59982.000000,47.000000,NaN,NaN,231.000000,45.000000,2024-10-01 00:00:00,8955.000000,NaN
max,80000.000000,80.000000,NaN,NaN,432.000000,50.000000,2025-12-31 00:00:00,15850.000000,NaN


# Download

In [ ]:
df_vendas.to_csv(
    "base_vendas_tratada.csv",
    index=False,
    encoding="utf-8-sig"
)
